In [1]:
import pandas as pd
import numpy as np
import time
from neo4j import GraphDatabase

In [2]:
df = pd.read_csv('Datasets\dataset_final.csv')

In [3]:
uri = "neo4j+s://b073ebf5.databases.neo4j.io"
user = "neo4j"
password = "dFUBgoUZAXirgfzSyJT4z_dpznMWBU-g5Z53SHGtonE"

driver = GraphDatabase.driver(uri, auth=(user, password))

# Test de conexión
def test_connection():
    with driver.session() as session:
        result = session.run("RETURN 'Conexión exitosa con AuraDB' AS mensaje")
        for record in result:
            print(record["mensaje"])

test_connection()

Conexión exitosa con AuraDB


In [4]:
def crear_nodo_depto(tx, datos):
    query = """
    CREATE (:Departamento {
        id: $id,
        latitude: $latitude,
        longitude: $longitude
    })
    """
    tx.run(query, **datos)
    
def cargar_deptos(df):
    with driver.session() as session:
        for _, row in df.iterrows():
            datos = {
                "id": row["id"],
                "latitude": float(row["latitud"]),
                "longitude": float(row["longitud"])
            }
            session.write_transaction(crear_nodo_depto, datos)

In [9]:
start = time.time()
cargar_deptos(df)
end = time.time()
print(f'Tiempo de ejecucion = {end-start} segundos')

C:\Users\Pc-ADS\AppData\Local\Temp\ipykernel_26040\759094799.py:19: DeprecationWarning: write_transaction has been renamed to execute_write
  session.write_transaction(crear_nodo_depto, datos)
[#D27A]  _: <CONNECTION> error: Failed to write data to connection ResolvedIPv4Address(('34.121.155.65', 7687)) (ResolvedIPv4Address(('34.121.155.65', 7687))): ConnectionResetError(10054, 'An existing connection was forcibly closed by the remote host', None, 10054, None)
Unable to retrieve routing information
Transaction failed and will be retried in 1.09967258644022s (Unable to retrieve routing information)


Tiempo de ejecucion = 10157.992450237274 segundos


In [18]:
df_pois = pd.read_csv('Datasets\pois_final.csv')
df_pois["poi_id"] = df_pois.index.astype('int64')
df_pois["name"] = df_pois["name"].fillna("NoName")
df_pois.sample(10)

,longitude,latitude,type,class,name,poi_id
3959,-70.674136,-33.396817,School,education_prim,Escuela Especial Reina María,3959
5400,-70.647873,-33.468365,School,education_prim,SaLa Cuna y Jardín Infantil Institución Sweet,5400
5229,-70.673289,-33.412314,School,education_prim,Escuela Básica Santa Teresita,5229
18762,-70.577722,-33.551605,School,education_prim,Escuela Basica Santa Ana De La Luz,18762
755,-70.661414,-33.489340,Butcher,food_and_drink_stores,Carnes RV,755
5670,-70.673193,-33.440730,Cafe,food_and_drink,Brunet,5670
17254,-70.773322,-33.528985,School,education_prim,Escuela Básica Los Bosquinos,17254
13216,-70.719755,-33.492898,Clinic,medical,Cesfam Dr. Norman Voulliéme,13216
3053,-70.519683,-33.349439,Swimming Pool,sport_and_leisure,Club de Campo Colegio Médico,3053
17033,-70.672546,-33.527230,Fitness,sport_and_leisure,NoName,17033


In [19]:
df_pois.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21444 entries, 0 to 21443
Data columns (total 6 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   longitude  21444 non-null  float64
 1   latitude   21444 non-null  float64
 2   type       21444 non-null  object 
 3   class      21444 non-null  object 
 4   name       21444 non-null  object 
 5   poi_id     21444 non-null  int64  
dtypes: float64(2), int64(1), object(3)
memory usage: 1005.3+ KB


In [22]:
def crear_nodos_pois(tx, row):
    query = """
    MERGE (p:POI {
        poi_id: $poi_id
    })
    SET p.name = $name,
        p.latitude = $latitude,
        p.longitude = $longitude,
        p.class = $class
    """
    tx.run(query, **row)

def cargar_pois(df):
    with driver.session() as session:
        for _, row in df.iterrows():
            datos = {
                "poi_id": row["poi_id"],
                "name": row["name"],
                "latitude": float(row["latitude"]),
                "longitude": float(row["longitude"]),
                "class": row["class"]
            }
            session.write_transaction(crear_nodos_pois, datos)

In [24]:
start = time.time()
cargar_pois(df_pois)
end = time.time()
print(f'Tiempo de ejecucion = {end-start} segundos')

C:\Users\Pc-ADS\AppData\Local\Temp\ipykernel_30280\3023171507.py:23: DeprecationWarning: write_transaction has been renamed to execute_write
  session.write_transaction(crear_nodos_pois, datos)
[#E2A8]  _: <CONNECTION> error: Failed to write data to connection ResolvedIPv4Address(('34.121.155.65', 7687)) (ResolvedIPv4Address(('34.121.155.65', 7687))): ConnectionResetError(10054, 'An existing connection was forcibly closed by the remote host', None, 10054, None)
Unable to retrieve routing information
Transaction failed and will be retried in 0.8504545047050933s (Unable to retrieve routing information)


Tiempo de ejecucion = 8770.718525409698 segundos
